# Aprendizaje Automático - Intermedio

## XGBoost

### Introducción

En este tutorial, aprenderemos a construir y optimizar modelos con **gradient boosting**. Este método domina muchas competiciones de Kaggle y logra resultados de vanguardia en una variedad de conjuntos de datos.

Durante gran parte de este curso hemos realizado predicciones con el método de random forest, que logra un mejor rendimiento que un solo árbol de decisión simplemente promediando las predicciones de muchos árboles de decisión.

Nos referimos al método de random forest como un "método de conjunto". Por definición, los **métodos de conjunto** combinan las predicciones de varios modelos (por ejemplo, varios árboles, en el caso de random forest).

A continuación, aprenderemos sobre otro método de conjunto llamado gradient boosting.

### Gradient Boosting

**Gradient boosting** es un método que pasa por ciclos para agregar modelos de forma iterativa a un conjunto.

Comienza por inicializar el conjunto con un solo modelo, cuyas predicciones pueden ser bastante ingenuas. (Incluso si sus predicciones son muy inexactas, las adiciones posteriores al conjunto abordarán esos errores).

Entonces, comenzamos el ciclo:

+ Primero, usamos el conjunto actual para generar predicciones para cada observación en el conjunto de datos. Para hacer una predicción, agregamos las predicciones de todos los modelos del conjunto.
+ Estas predicciones se utilizan para calcular una función de pérdida (como el [error cuadrático medio](https://en.wikipedia.org/wiki/Mean_squared_error), por ejemplo).
+ Luego, usamos la función de pérdida para ajustar un nuevo modelo que se agregará al conjunto. Específicamente, determinamos los parámetros del modelo para que agregar este nuevo modelo al conjunto reduzca la pérdida. (Nota al margen: *el "gradient" en "gradient boosting" se refiere al hecho de que usaremos el [descenso de gradiente](https://en.wikipedia.org/wiki/Gradient_descent) en la función de pérdida para determinar los parámetros en este nuevo modelo*).
+ Finalmente, agregamos el nuevo modelo al conjunto y ...
+ ... ¡repetir!

![gradient_boosting](./images/gradient_boosting.png)

### Ejemplo

Comenzamos cargando los datos de entrenamiento y validación en `X_train`, `X_valid`, `y_train` e `y_valid`.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leer datos
data = pd.read_csv('./input/melbourne-housing-snapshot/melb_data.csv')

# Selecciona subconjunto de predictores
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Selecciona objetivo
y = data.Price

# Separa los datos en conjutos de entrenamientos y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

En este ejemplo, trabajaremos con la biblioteca XGBoost. **XGBoost** significa **extreme gradient boosting**, que es una implementación de gradient boosting con varias características adicionales centradas en el rendimiento y la velocidad. (*Scikit-learn tiene otra versión de gradient boosting, pero XGBoost tiene algunas ventajas técnicas*).

Importamos la API scikit-learn para XGBoost (`xgboost.XGBRegressor`). Esto nos permite construir y ajustar un modelo tal como lo haríamos en scikit-learn. Como verás en la salida, la clase `XGBRegressor` tiene muchos parámetros ajustables: ¡pronto los aprenderás!

In [ ]:
from xgboost import XGBRegressor
my_model = XGBRegressor()

In [ ]:
my_model.fit(X_train, y_train)

También hacemos predicciones y evaluamos el modelo.

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

### Ajuste de parámetros

XGBoost tiene algunos parámetros que pueden afectar dramáticamente la precisión y la velocidad de entrenamiento. Los primeros parámetros que se deben comprender son:

`n_estimators`

`n_estimators` especifica cuántas veces pasar por el ciclo de modelado descrito anteriormente. Es igual al número de modelos que incluimos en el conjunto.

+ Un valor demasiado *bajo* provoca *underfitting*, lo que conduce a predicciones inexactas tanto en los datos de entrenamiento como en los de prueba.
+ Un valor demasiado *alto* provoca un *overfitting*, lo que genera predicciones precisas sobre los datos de entrenamiento, pero predicciones inexactas sobre los datos de las pruebas (que es lo que nos importa).

Los valores típicos oscilan entre 100 y 1000, aunque esto depende mucho del parámetro `learning_rate` que se analiza a continuación.

Aquí está el código para establecer el número de modelos en el conjunto:

In [ ]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

`early_stopping_rounds`
`early_stopping_rounds` ofrece una manera de encontrar automáticamente el valor ideal para `n_estimators`. *Early stopping* hace que el modelo deje de iterar cuando la puntuación de validación deja de mejorar, incluso si no estamos en la parada indicada para `n_estimators`. Es inteligente establecer un valor alto para `n_estimators` y luego usar `early_stopping_rounds` para encontrar el momento óptimo para dejar de iterar.

Dado que la probabilidad aleatoria a veces causa una sola ronda en la que los puntajes de validación no mejoran, se debe especificar un número para cuántas rondas de deterioro directo se permiten antes de detenerse. Establecer `early_stopping_rounds = 5` es una opción razonable. En este caso, nos detenemos después de 5 rondas consecutivas de puntuaciones de validación deteriorados.

Al usar `early_stopping_rounds`, también se deben reservar algunos datos para calcular las puntuaciones de validación; esto se hace configurando el parámetro `eval_set`.

Podemos modificar el ejemplo anterior para incluir la early stopping:

In [ ]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

Si más tarde se desea ajustar un modelo con todos sus datos establezca `n_estimators` en el valor que considere óptimo cuando se ejecuta con detención temprana.

`learning rate`

En lugar de obtener predicciones simplemente sumando las predicciones de cada modelo de componente, podemos multiplicar las predicciones de cada modelo por un pequeño número (conocido como la tasa de aprendizaje p **learning rate**) antes de sumarlas.

Esto significa que cada árbol que agreguemos al conjunto nos ayuda menos. Por lo tanto, podemos establecer un valor más alto para `n_estimators` sin overfitting. Si utilizamos early stopping, la cantidad apropiada de árboles se determinará automáticamente.

En general, una pequeña tasa de aprendizaje y un gran número de estimadores producirán modelos XGBoost más precisos, aunque también llevará más tiempo entrenar el modelo, ya que realiza más iteraciones a lo largo del ciclo. De forma predeterminada, XGBoost establece `learning_rate = 0.1`.

Al modificar el ejemplo anterior para cambiar la tasa de aprendizaje, se obtiene el siguiente código:

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

`n_jobs`

En conjuntos de datos más grandes donde el tiempo de ejecución es una consideración, se puede usar el paralelismo para construir nuestros modelos más rápido. Es común establecer el parámetro `n_jobs` igual al número de núcleos en su máquina. En conjuntos de datos más pequeños, esto no ayudará.

El modelo resultante no será mejor, por lo que la micro-optimización para el tiempo de adaptación generalmente no es más que una distracción. Pero es útil en grandes conjuntos de datos en los que, de lo contrario, pasaría mucho tiempo esperando durante el comando `fit`.

Aquí está el ejemplo modificado:

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

### Conclusión

[XGBoost](https://xgboost.readthedocs.io/en/latest/) es una librería de software líder para trabajar con datos tabulares estándar (el tipo de datos que se almacena en Pandas DataFrames, en oposición a tipos de datos más exóticos como imágenes y videos). Con un ajuste cuidadoso de los parámetros, se pueden entrenar modelos de alta precisión.